In [1]:
# =============================
# 1. Imports
# =============================
import psycopg2
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings
import os

In [3]:
# =============================
# 2. Connect to Postgres
# =============================
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT")),
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
}
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

# Example: fetch first 100 rows for testing
cur.execute("SELECT random_key, persian_name FROM base_products LIMIT 100")
rows = cur.fetchall()

In [4]:
# Choose embedding model
# - text-embedding-3-small → 1536 dims
# - text-embedding-3-large → 3072 dims
embedding_model = "text-embedding-3-small"
dim = 1536 if "small" in embedding_model else 3072

# Create / open a Qdrant database on disk
client = QdrantClient(path="./qdrant_db_test")   # any folder path works

# Create collection (example with OpenAI small embeddings)
client.recreate_collection(
    collection_name="products",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)



C:\Users\Behnam_Local\AppData\Local\Temp\ipykernel_1512\1174794956.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [5]:
# =============================
# 4. Initialize embeddings + QdrantVectorStore
# =============================
OPENAI_API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")
embeddings = OpenAIEmbeddings(model=embedding_model, 
                              api_key=OPENAI_API_KEY, base_url=BASE_URL)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="products",
    embedding=embeddings,
)

In [5]:
# =============================
# 5. Insert rows into Qdrant
# =============================
texts = [row[1] for row in rows]          # persian_name
metadatas = [{"random_key": row[0], "persian_name": row[1]} for row in rows]

vector_store.add_texts(texts=texts, metadatas=metadatas)

print(f"Inserted {len(texts)} rows into Qdrant.")

Inserted 100 rows into Qdrant.


In [3]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from openai import OpenAI
from dotenv import load_dotenv

import os
import psycopg2
from psycopg2.extras import RealDictCursor


load_dotenv()

DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT", 5432)),
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
}
# -------------------------
# CONFIG
# -------------------------
QDRANT_PATH = "../data/qdrant_db"  # local disk folder
COLLECTION_NAME = "products"
EMBED_MODEL = "text-embedding-3-small"
OPENAI_API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")
TOP_K = 5

# Load API credentials
OPENAI_API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")

# Initialize embeddings
embeddings = OpenAIEmbeddings(
    model=EMBED_MODEL,
    api_key=OPENAI_API_KEY,
    base_url=BASE_URL,
)

# Connect to local disk-based Qdrant
client = QdrantClient(path=QDRANT_PATH)

# Wrap with LangChain VectorStore
vector_store = QdrantVectorStore(
    client=client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
)


In [12]:
def get_persian_name_from_db(random_key):
    """
    Fetch the Persian name from the database using the random_key.
    
    Returns the name as a string if found, otherwise returns None.
    """
    if not random_key:
        return None

    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor(cursor_factory=RealDictCursor)

        # Replace 'your_table' and 'persian_name_column' with actual table/column names
        query = """
            SELECT persian_name
            FROM base_products
            WHERE random_key = %s
            LIMIT 1;
        """
        cursor.execute(query, (random_key,))
        result = cursor.fetchone()

        cursor.close()
        conn.close()

        if result and "persian_name" in result:
            return result["persian_name"]
        return None

    except Exception as e:
        print(f"Error querying Persian name for random_key {random_key}: {e}")
        return None


def retrieve_from_vector_db(query: str, top_k: int = TOP_K):
    results = vector_store.similarity_search(query, k=top_k)
    values = []
    for _, doc in enumerate(results, start=1):
        print(doc)
        meta_data = {}
        name = get_persian_name_from_db(doc.metadata.get("random_key"))
        rk = doc.metadata.get("random_key")
        meta_data['persian_name'] = name
        meta_data['random_key'] = rk
        values.append(meta_data)
    return values


In [13]:
vs = retrieve_from_vector_db("ساعت دیواری")

page_content='ساعت دیواری suho3' metadata={'random_key': 'aacsio', '_id': '18808e4f32174b43b751b67669b2dafa', '_collection_name': 'products'}
page_content='ساعت‌های دیواری راویتا طرح 3100' metadata={'random_key': 'aaaeyv', '_id': '1eadfd1e3dc04880a8cb4eff5166d0aa', '_collection_name': 'products'}
page_content='ساعت دیواری رزینی مولتی دار سیلور - قطر ۶۰' metadata={'random_key': 'aacwrs', '_id': 'a51b51a43897476ea12af07cead0b48c', '_collection_name': 'products'}
page_content='شومینه دیواری برقی شومینه بیستون مدل H.D_90_L2.bs' metadata={'random_key': 'aabwzx', '_id': '1128ef5f26b7427bb6468297d84d0aa5', '_collection_name': 'products'}
page_content='سردوشی برگی انجیری طلایی سنگ و ملیله | فروشگاه اینترنتی گالری ویان' metadata={'random_key': 'aaauyt', '_id': '5c912d06fe3c45798774dfb0c70e300c', '_collection_name': 'products'}


In [11]:
vs

[{'persian_name': 'ساعت دیواری suho3', 'random_key': 'aacsio'},
 {'persian_name': 'ساعت\u200cهای دیواری راویتا طرح 3100',
  'random_key': 'aaaeyv'},
 {'persian_name': 'ساعت دیواری رزینی مولتی دار سیلور - قطر ۶۰',
  'random_key': 'aacwrs'},
 {'persian_name': 'شومینه دیواری برقی شومینه بیستون مدل H.D_90_L2.bs',
  'random_key': 'aabwzx'},
 {'persian_name': 'سردوشی برگی انجیری طلایی سنگ و ملیله | فروشگاه اینترنتی گالری ویان',
  'random_key': 'aaauyt'}]